In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install poutyne
!pip install git+https://github.com/ulaval-damas/glo4030-labs.git

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import poutyne as pt
from deeplib.history import History
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import v2

In [ ]:
# Définition des transformations pour les données à partir imageNet
transform1 = v2.Compose([
    v2.Resize((224, 224)),
    v2.ToImage(), 
    v2.ToDtype(torch.float32, scale=True),
    
    v2.RandomVerticalFlip(0.5),
    v2.ColorJitter(),
    v2.GaussianBlur(kernel_size=5),
    
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = v2.Compose([
    v2.Resize((224, 224)),
    v2.ToImage(), 
    v2.ToDtype(torch.float32, scale=True),
    
    
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
batch_len = 40
transform = transform1

dataset = torchvision.datasets.ImageFolder('/kaggle/input/glo-7030-ou-suis-je-h2024/train', transform=transform)

In [ ]:
a,b = dataset[-1]

print(len(dataset))
print(b)

In [ ]:
# la classe ImageFolder assigne automatiquement un label pour chaque nom de classe (class -> idx)
print('class -> idx : ',dataset.class_to_idx)

# on aura besoin d'un dictionnaire qui fait le sens inverse (idx -> class)
idx_to_class = {dataset.class_to_idx[class_name]: class_name for class_name in  dataset.class_to_idx}
print('idx -> class : ',idx_to_class)

In [ ]:
TRAIN_SIZE = 0.9
train_set, valid_set = random_split(dataset, [TRAIN_SIZE, 1-TRAIN_SIZE])

# train_set.transform = transform1
# valid_set.transform = test_transform

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_len, shuffle=True)#,collate_fn=collate_fn)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_len, shuffle=False)

In [ ]:
print(len(train_set))
print(len(valid_set))
print(len(dataset) , " et :", len(train_set)+len(valid_set))

In [ ]:
class EFA_Net(nn.Module):
  def __init__(self, NUM_CLASSES):
    super().__init__()

    model = models.efficientnet_v2_l(weights="DEFAULT")
    model.classifier[-1] = nn.Linear(in_features=model.classifier[-1].in_features, out_features= NUM_CLASSES, bias = True)

#     for name, param in model.named_parameters():
#       print(name)

    # print(model)
    
    for n,p in model.named_parameters():
      if 'classifier' not in n and 'features.8' not in n and 'features.7.6' not in n and 'features.7.5' not in n:
            
        p.requires_grad = False
      else :
        print(n)

    self.model = model

  def forward(self, x):
    return self.model(x)


In [ ]:
#Paramètres
NUM_CLASSES = 5
N_EPOCHS = 100
TRAIN_SIZE = 0.8
USE_GPU = True

lr = 0.1#0.006

#Initialisations
net = EFA_Net(NUM_CLASSES)

# optimizer = optim.SGD(net.parameters(), lr=lr)
# optimizer = optim.SGD(net.parameters(), lr=0.01, weight_decay=1e-3, nesterov=True, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=lr)

loss = nn.CrossEntropyLoss()


In [ ]:
# On crée un objet Model de Poutyne avec la fonction de perte et les métriques souhaitées.
model = pt.Model(net, optimizer, 'cross_entropy', batch_metrics=['accuracy'])
model.cuda()

# Comme dit précedemment, on utilise la méthode `fit_generator` pour procéder à l'entraînement.
p=7

scheduler = pt.ReduceLROnPlateau(monitor='val_acc', mode='max', patience=3, factor=0.5, verbose=True)

# Early stopping sous la forme d'un Callback
early_stopping = pt.EarlyStopping(monitor='val_acc', mode='max', min_delta=1e-5, patience=p, verbose=True)

# history = train(net, optimizer, cifar_train, n_epoch=40, batch_size=64, callbacks=[scheduler, early_stopping], use_gpu=True)

history = model.fit_generator(train_loader, valid_loader, epochs=N_EPOCHS, callbacks=[scheduler, early_stopping])

# La classe History peut aussi prendre un historique de Poutyne en entrée.
History(history).display()


In [ ]:
# on hérite de la classe Dataset de PyTorch 
class TestDataset(Dataset):
    def __init__(self, test_path, transform=None):
        
        # lister toutes les images dans le repertoire test
        # self.image_paths va etre une liste contenant des 
        # elements de type Path (pour plus d'info voir la class Path de pathlib)
        # cela fonctionne seulement pour les versions de python >3.4
        self.image_paths = list(pathlib.Path(test_path).glob('*.jpg'))
        
        # trier les noms des fichiers d'images par ordre
        self.image_paths.sort()
        
        # garder la fonctions de tranform dans self pour l'utiliser dans __getitem__
        self.transform = transform

    def __getitem__(self, index):
        # index est un nombre qui vient du dataloader (il est entre 0 et ce que retourne la methode __len__ ci-dessous)
        # c'est donc entre 0 et le nombre d'images de test
        img_path = self.image_paths[index]
        
        # retourner aussi le nom de l'image en question(i.e. '0xxxx')
        # sans l'extension '.jpg', donc on ignore les 4 dernier caracteres du nom de l'image (.jpg) avec [:-4]
        # par exemple, pour l'image '00037.jpg' on retourne '00037'
        # cela va vous etre util pour generer le fichier predictions.csv
        img_name = img_path.name[:-4] #img_path est un objet de type Path, et a donc un attribut 'name'
        
        # lire l'image avec PIL
        img = Image.open(img_path)
        
        # appliquer les transforms s'il y'en a
        if self.transform is not None:
            img = self.transform(img)
        
        return img, img_name

    def __len__(self):
        return len(self.image_paths)

In [ ]:
# nous utilisons pathlib afin de parcourir les dossiers et fichiers
import pathlib
from PIL import Image 

In [ ]:
# Nous allons tester la classe TestDataset
test_dataset = TestDataset('/kaggle/input/glo-7030-ou-suis-je-h2024/test', transform=test_transform) # ici je transforme les images en Tensor pour une utilisation rapide de PyTorch Dataloader 

# on aura besoin d'un dataloader qui enveloppe notre objet test_dataset
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4) # souvent le shuffle est mis a False, mais avec notre implementation, ca marche dans tous les cas

In [ ]:
# generons une prediction aleatoire pour notre test
label_predictions = []
image_names = []
for batch,im_names in test_loader:
    # remplir avec des predictions aleatoires (entre 0 et 4)
    random_preds = model.predict(batch,verbose=False)
#     print(random_preds)
    _, preds = torch.max(torch.Tensor(random_preds), 1)
#     preds = m(torch.Tensor(random_preds))
#     print(preds)
    label_predictions.extend(preds) 
    
    # retenir les noms des images
    image_names.extend(im_names) 

assert len(label_predictions) == len(image_names) 
assert len(label_predictions) == len(test_dataset) # est-ce qu'on a prédits tous les exemples de tests ?
print(f'Il y a {len(label_predictions)} exemples de test')

In [ ]:
import pandas as pd

In [ ]:
# Utilisons Pandas afin de generer un DataFrame

predictions_df = pd.DataFrame(data=zip(image_names, label_predictions), columns=['image_name', 'class_label'])
predictions_df

In [ ]:
predictions_df['class_label']=predictions_df['class_label'].apply(lambda x : int(x))
predictions_df

In [ ]:
predictions_df['class']=predictions_df['class_label'].apply(lambda x : idx_to_class[x])
predictions_df

In [ ]:
# on drop la colonne class_label avant d'entregistrer en fichier CSV
predictions_df = predictions_df.drop(labels=['class_label'], axis=1)
predictions_df

In [ ]:
# enregistrer dans fichier de format CSV
# N'OUBLIEZ PAS DE METTRE index=None
predictions_df.to_csv('./mes_predictions.csv', index=None)